<a href="https://colab.research.google.com/github/Weizhuo-Zhang/ML_coursera/blob/master/hands_on_ML/8_dimensionality_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 八、降维

很多机器学习的问题都会涉及到几千甚至数百万维的特征的训练实例。这叫做维数灾难(Curse of dimentionality)。

**警告：**

降维肯定会丢失一些信息（这就好比将一个图片压缩成JPEG的格式会降低图片的质量）

## 降维的主要方法

## 投影(Projection)

## 流行学习(Manifold learning)

## 主成分分析(PCA) Principal Component Analysis

### 保留(最大)方差

### 主成分(Principal Components)

奇异值分解(SVD)的标准矩阵分解技术，可以找到训练集的主成分。

下面的Python代码使用了Numpy提供的`svd()`函数获得训练集的所有主成分，然后提取前两个PC：

```python
X_centered = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X_centered)
c1 = V.T[:,0]
c2 = V.T[:,1]
```

**警告：**

PCA假定数据集以原点为中心。如果自己实现PCA，不要忘记首先要对数据做中心化处理。

### 投影到 d 维空间

一旦确认了所有的主成分，就可以将数据集投影到由前 `d` 个成分构成的超平面上，从而将数据集的维数降低至 `d` 维，选择的这个超平面可以确保留尽可能多的方差。

**公式：** 将训练集投影到 `d` 维空间：
$$X_{d-proj} =  X\cdot W_d$$


**代码**
```python
W2 = V.T[:, :2]
X2D = X_centered.dot(W2)
```

### 使用 Scikit-Learn

**`Scikit-Learn`** 的 PCA 类使用 SVD 分解来实现。以下代码应用 PCA 将数据集的维度降低至两维

```python
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D=pca.fit_transform(X)
```

PCA，可以使用 `components_` 访问每一个主成分（注意，它返回以 PC 作为水平向量的矩阵，因此，如果我们想要获得第一个主成分则可以写成 `pca.components_.T[:,0]` ）

### 方差解释率(Explained Variance Ratio)

另一个非常有用的信息是每个主成分的方差解释率，可以通过 `explained_variance_ratio_` 变量获得。它表示位于每个主成分轴上的数据集方差的比例。

```python
>>> print(pca.explained_variance_ratio_)
array([0.845433, 0.143422])
```

这表明，84.5%的数据集方差位于第一轴，14.3%的方差位于第二轴。第三周比例小，可能没有包含什么信息。

### 选择正确的维度

下面的代码在不降维的情况下进行 PCA，然后计算出保留训练集方差 95% 所需的最小维数：

```python
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
```

可以设置 `n_components = d` 再次运行 PCA。但是，有一个更好的选择：不指定想要保留的主成分个数，而是将 `n_components` 设置为0.0 到1.0 之间的浮点数，表明您希望保留的方差比率：
```python
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)
```

PCA 压缩